In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,...,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated,TARGET
count,7.603800e+04,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,...,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000
mean,4.994761e+07,6.148886,1.580289,10.118296,3.703661,11.320221,4.404127,3.705805,1.022344,2.145335,...,0.112365,0.116152,0.161130,0.085484,0.059733,0.367803,0.449025,0.183172,0.052224,0.467095
std,2.890145e+07,7.233436,1.814161,15.353455,4.089925,10.480493,3.341382,2.615952,0.150621,1.021198,...,0.315817,0.320410,0.367653,0.279602,0.236994,0.482211,0.497398,0.386810,0.222480,1.612194
min,2.100000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483719e+07,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,2.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990721e+07,4.000000,1.000000,3.000000,2.000000,8.000000,3.000000,3.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.510112e+07,10.000000,2.000000,14.000000,7.000000,16.000000,6.000000,5.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,9.999863e+07,89.000000,12.000000,84.000000,12.000000,43.000000,27.000000,19.000000,3.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000


In [3]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

X = data.loc[:, data.columns != 'TARGET']
y = data['TARGET']
y.describe()

count    76038.000000
mean         0.467095
std          1.612194
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         15.000000
Name: TARGET, dtype: float64

In [5]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    pred_y = pred_y.astype(int)
    # The root mean square logarithmic error
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    
    print ("RMSLE: %.4f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(pred_y, y_test)
    print ("R2 value: %.4f" % r2)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=99)

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('regr', Lasso())
])

param_grid = [
    {
        'regr': [Lasso(), Ridge()],
        'regr__alpha': np.logspace(-4, 1, 6),
    },
    {
        'regr': [SGDRegressor()],
        'regr__alpha': np.logspace(-5, 0, 6),
        'regr__max_iter': [500, 1000],
    },
    {
        'regr': [linear_model.LinearRegression()]
    }
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

eval(grid, X_test, y_test)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.0001 
[CV] regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.0001 
[CV] regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.0001 
[CV] regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.001 
[CV] regr=Lasso(alpha=1.0, copy_X=True, fit_intercep

   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.001 
[CV]  regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.0001, total=   0.8s
[CV] regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.001 
[CV]  regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.001, total=   0.7s
[CV] regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.01 
[CV]  regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.001, total=   9.3s
[CV]  reg

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.0s


[CV]  regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=0.1, total=   0.9s
[CV] regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=10.0 
[CV]  regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=1.0, total=   0.9s
[CV] regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=10.0 
[CV]  regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=1.0, total=   0.9s
[CV] regr=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), regr__alpha=10.0 
[CV]

/Users/truongkhanhduy/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


...

[CV]  regr=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), regr__alpha=0.0001, total=  40.3s
[CV] regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=0.0001, regr__max_iter=500 
[CV]  regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=1e-05, regr__max_iter=500, total=  28.9s
[CV] regr=SGDRegressor(alpha=0.0001

       warm_start=False), regr__alpha=0.0001, regr__max_iter=1000, total=  39.4s
[CV] regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=0.01, regr__max_iter=500 
[CV]  regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=0.0001, regr__max_iter=1000, total=  39.9s
[CV]  regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n

       warm_start=False), regr__alpha=0.1, regr__max_iter=1000 [CV]  regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=0.1, regr__max_iter=500, total=  29.3s

[CV]  regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False), regr__alpha=0.01, regr__max_iter=1000, total=  52.0s
[CV] regr=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=

[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  3.4min finished


RMSLE: 0.4996
R2 value: -20.6277


In [19]:
predicted = grid.predict(X_test)

print('Score:\t{}'.format(grid.score(X_test, y_test)))

Score:	0.0735334162321557


In [21]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']

eval(grid, X_submit, y_submit)

RMSLE: 0.4892
R2 value: -14.7400
